## Number of synapses per connection

First, some packages needs to be installed and imported.

In [ ]:
import bluepy
from bluepy.v2 import Cell
import pandas
import matplotlib.pyplot as plt 
import numpy as np
import seaborn

Reading and preparing the data

In [ ]:
circuit_path = '/mnt/circuits/O1/20181114/CircuitConfig'
circ = bluepy.Circuit(circuit_path)

In [ ]:
# nsample = 100
mtypes = circ.v2.cells.mtypes
model_mean = pandas.DataFrame(index=mtypes, columns=mtypes.union({u'PV'}), dtype=float)
model_std = pandas.DataFrame(index=mtypes, columns=mtypes.union({u'PV'}), dtype=float)

In the following cell, ...

In [ ]:
for pre_mtype in mtypes:
    for post_mtype in mtypes:

        pre = circ.v2.cells.ids(group={Cell.MTYPE: pre_mtype, Cell.REGION: '@mc2.*'})
        
        post = circ.v2.cells.ids(post_mtype)
        data = np.array([p[2] for p in circ.v2.connectome.iter_connections(pre, post, return_synapse_count=True)])
        # note the syntax dataframe[column][row], so we have table[post][pre]
        model_mean[post_mtype][pre_mtype] = data.mean()
        model_std[post_mtype][pre_mtype] = data.std()
    print pre_mtype

In [ ]:
pre_mtype = 'SP_PVBC'
post_mtype = 'PV'

pre = circ.v2.cells.ids(group={Cell.MTYPE: pre_mtype, Cell.REGION: '@mc2.*'})      
post = circ.v2.cells.ids('PV')
data = np.array([p[2] for p in circ.v2.connectome.iter_connections(pre, post, return_synapse_count=True)])
# note the syntax dataframe[column][row], so we have table[post][pre]
model_mean[post_mtype][pre_mtype] = data.mean()
model_std[post_mtype][pre_mtype] = data.std()

In [ ]:
ax = seaborn.heatmap(model_mean)

fig = plt.gcf()
fig.suptitle('Number of synapses per connection', )

ax.set_xlabel('presynaptic mtype')
ax.set_ylabel('postsynaptic mtype')

ax.collections[0].colorbar.set_label("# synapses")

fig.show()

In [ ]:
# Pull data to kernel to be used
# !pip install -q --upgrade --force-reinstall "hbp-service-client"
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
clients.storage.download_file(collab_path + '/nsyn_per_connection_20190131.tsv', 'nsyn_per_connection_20190131.tsv')

In [ ]:
# reference biological dataset
bio_path = 'nsyn_per_connection_20190131.tsv'

In [ ]:
df = pandas.read_csv(bio_path, skiprows=1, names=['pre', 'post', 'bio_mean', 'bio_std'], usecols=[0, 1, 2, 3], delim_whitespace=True)
df.head()

In [ ]:
df['mod_mean'] = np.NAN
df['mod_std'] = np.NAN

In [ ]:
for idx in df.index:
    pre = df.loc[idx, 'pre']
    post = df.loc[idx, 'post']
    df.loc[idx, 'mod_mean'] = model_mean[post][pre]
    df.loc[idx, 'mod_std'] = model_std[post][pre]

In [ ]:
plt.clf
x = df['mod_mean'].values
y = df['bio_mean'].values
l = np.linspace(0, max(x.max(), y.max()), 50)
fig, ax = plt.subplots()
fig.suptitle('synapses per connection')
ax.plot(x, y, 'o')
ax.errorbar(x, y, xerr=df['mod_std'].values, yerr=df['bio_std'].values, fmt='o', ecolor='g', capthick=2)
ax.plot(l, l, 'k--')
ax.set_xlabel('Model (#)')
ax.set_ylabel('Experiment (#)')

fig.show()

### Exercise #1
Calculate the average number of synapses per connection in t he four classes of connections (EE, EI, IE, II). Put the answer in a list called ans_3. Respect the order (EE, EI, IE, II).

### Exercise #2
Calculate the distribution of number of synapses from SP_PVBC to SP_PC. Put the answer in a list named ans_4 with mean and std

In [ ]:
# Work here

In [ ]:
# This is to generate the answers to paste in the submission box below.
# Run this and copy-paste the output into the box below
print json.dumps(dict([("ans_3", ans_3),
                       ("ans_4", ans_4)]))

In [ ]:
!pip -q install -i https://bbpteam.epfl.ch/repository/devpi/simple/ single-cell-mooc-client==0.0.5 
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission(hideToken=True)

In [ ]:
s.show_submission(ROLLBACK, TOKEN)